# Introduction

In this notebook, we will experiment with how user queries are handled in our veterinary information retrieval system. Several collections have already been set up in the Chroma database, allowing us to directly perform information retrieval without additional setup. This environment enables us to test and refine the process of transforming user input into actionable queries and retrieving relevant information from our knowledge base.

In [27]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_chroma import Chroma

persist_directory = '../chroma/textbook_test_Nutrition'
id_key = "doc_id"

open_clip_embeddings = OpenCLIPEmbeddings(model_name="ViT-g-14", checkpoint="laion2b_s34b_b88k")

# Vectorstore for summaries (for similarity search)
vectorstore = Chroma(
    collection_name="summaries",
    persist_directory=persist_directory,
    embedding_function=open_clip_embeddings
)
# Persistent docstore for originals (all modalities)
docstore = Chroma(
    collection_name="originals",
    persist_directory=persist_directory,
    embedding_function=open_clip_embeddings
)

# Instantiate the retriever

class UnifiedRetriever:
        
        def __init__(self, vectorstore, docstore, id_key="doc_id"):
            self.vectorstore = vectorstore
            self.docstore = docstore
            self.id_key = id_key
            self._collection = docstore._collection

        def retrieve(self, query, k=5):
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            output = []
            for doc, score in results:
                doc_id = doc.metadata.get(self.id_key)
                try:
                    original = self._collection.get(ids=[doc_id], include=["documents", "metadatas"])
                    original_doc = original["documents"][0] if original["documents"] else None
                    original_meta = original["metadatas"][0] if original["metadatas"] else None
                except Exception as e:
                    original_doc = None
                    original_meta = None
                output.append({
                    "summary": doc.page_content,
                    "original": original_doc,
                    "original_metadata": original_meta,
                    "summary_metadata": doc.metadata,
                    "score": score
                })
            return output

retriever = UnifiedRetriever(vectorstore, docstore, id_key=id_key)

# Handling User Input: Analysis Image

Let's use a image of a under weighted cat. This cat is considerablly skinny with bones showing. 

  ![Skinney Cat](./skinny_cat.jpg)

In [28]:
query = "What's going on with my cat? What should I do?" 
image_path = "./skinny_cat.jpg"

import base64
import ollama

# --- Configuration for the image ---
# IMPORTANT: Adjust this path if your cat.jpg is in a different location
# image_model = "minicpm-v:8b" # Or "llava:7b" or another suitable vision model you have installed via Ollama
# image_model = "llava:7b" # 
image_model = "llama3.2-vision" # 

# --- 1. Generate a textual summary of the image using an LLM ---
print(f" ⏳ Processing image: {image_path}")

image_summary = "Could not generate image summary." # Default in case of error
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}. Please check the path.")
else:
    try:
        # Read and encode image in base64
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')

        # Updated prompt for detailed image summarization
        image_summarization_prompt = """From a feline veterinary stand point, provide a highly detailed and objective 
                description of the image. Focus on all observable elements, actions, 
                objects, subjects, their attributes (e.g., color, size, texture), 
                their spatial relationships, and any discernible context or implied scene. 
                Also focus on all possible health issue.
                Describe any text present in the image. This description must be exhaustive 
                and purely factual, capturing every significant visual detail to serve as a 
                comprehensive textual representation for further analysis by another AI model. 
                If the image is entirely irrelevant or contains no discernible subject, 
                state "No relevant visual information."."""

        # Send image to ollama for vision model processing
        response = ollama.chat(
            model=image_model,
            messages=[
                {
                    'role': 'user',
                    'content': image_summarization_prompt,
                    'images': [image_data]
                }
            ]
        )
        image_summary = response['message']['content']
        print("--- Generated Image Summary ---")
        print(image_summary)

    except Exception as e:
        print(f"Error processing image with Ollama: {e}")

# This 'image_summary' can now be used along with your user's text query
# for retrieval or further processing in your RAG pipeline.

 ⏳ Processing image: ./skinny_cat.jpg


KeyboardInterrupt: 

# Handling User Input: Refine Query

In [6]:
import uuid
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the LLM for query refinement (using the same model as your RAG chain if appropriate)
query_refinement_model = ChatOllama(model="llama3.2")

# Prompt for query refinement
query_refinement_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to rephrase and expand the given user query \
into a more detailed and context-rich query that can be used to retrieve relevant information \
from a veterinary knowledge base. Use the provided image description to add visual context \
and relevant keywords to the refined query. Focus on adding relevant keywords, clarifying intent, \
and anticipating related information that might be helpful. The output should be a single, refined query.

Original query: {original_query}
Image description: {image_summary}"""
)

# Create the query refinement chain
query_refinement_chain = (
    {
        "original_query": RunnablePassthrough(),
        "image_summary": RunnablePassthrough()
    }
    | query_refinement_prompt
    | query_refinement_model
    | StrOutputParser()
)# --- Demonstration of query refinement and then retrieval with scores ---


print(f"Original user query: {query}")
print(f"Image Summary: {image_summary}")

refined_query = query_refinement_chain.invoke(
    {"original_query": query, "image_summary": image_summary}
)

print("-"*80)
print(f"Refined query: {refined_query}")


Original user query: What's going on with my cat? What should I do?
Image Summary: The image depicts a cat standing next to a bowl, with its head resting on the edge of the bowl. The cat is positioned on a tiled floor, and its body is oriented towards the right side of the image.

* **Cat**
	+ The cat is standing on the floor, with its head resting on the edge of the bowl.
	+ Its body is oriented towards the right side of the image.
	+ The cat appears to be thin and underweight, with a noticeable lack of muscle mass.
	+ Its fur is light-colored, and it has a short, smooth coat.
	+ The cat's ears are perked up, and its eyes are looking directly at the camera.
* **Bowl**
	+ The bowl is placed on the floor, next to the cat.
	+ It is a light-colored, plastic bowl with a smooth, rounded shape.
	+ The bowl appears to be empty, with no food or water visible inside.
* **Floor**
	+ The floor is made of small, square tiles that are arranged in a grid pattern.
	+ The tiles are a light-colored, be

# Query Decomposition

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
# Prompt for query decomposition
query_decomposition_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to break down the given complex query \
into a list of simpler, focused sub-queries. Each sub-query should be a standalone question \
that can be used to retrieve specific information from a veterinary knowledge base. \
Present the sub-queries as a JSON array of strings, where each string is a sub-query. \

Complex query: {refined_query}"""
)

# Create the query decomposition chain
query_decomposition_chain = (
    query_decomposition_prompt  
    | query_refinement_model    
    | JsonOutputParser() 
)

# --- Demonstration of query decomposition ---

print(f"Original refined query: {refined_query[:300]} ....")

decomposed_queries = query_decomposition_chain.invoke({"refined_query": refined_query})

print("-" * 80)
# print(f"Decomposed queries:\n{decomposed_queries}")

decomposed_queries[0]

Original refined query: Here's a refined query incorporating relevant keywords, clarifying intent, and anticipating related information that might be helpful:

"Given the image of a thin, underweight cat (Felis catus) standing next to an empty, light-colored plastic bowl on a clean ceramic tile floor, I am concerned about  ....
--------------------------------------------------------------------------------


'What are the common signs of underweight cats in veterinary medicine, including potential causes and consequences?'

# Contextual Retrievals 

In [15]:
class SummaryToOriginalRetriever:
    def __init__(self, vectorstore_summaries, vectorstore_original_texts, vectorstore_original_images):
        self.vectorstore = vectorstore_summaries
        self.textstore = vectorstore_original_texts
        self.imagestore = vectorstore_original_images

    def retrieve(self, query, k=5):
        # Search summaries
        results = self.vectorstore.similarity_search_with_score(query, k=k)
        originals = []
        for doc, score in results:
            doc_id = doc.metadata.get('doc_id')
            doc_type = doc.metadata.get('type', 'text')  # You may need to adjust this
            # Try to fetch from textstore first, then imagestore
            if doc_type == 'image':
                original = self.imagestore.get(doc_id)
            else:
                original = self.textstore.get(doc_id)
            originals.append({
                "summary": doc.page_content,
                "original": original,
                "score": score,
                "doc_id": doc_id,
                "type": doc_type
            })
        return originals

In [16]:
retriever = SummaryToOriginalRetriever(
    vectorstore_summaries,
    vectorstore_original_texts,
    vectorstore_original_images
)
results = retriever.retrieve(decomposed_queries[0], k=5)

for res in results:
    print(f"Summary: {res['summary'][:100]}...")
    print(f"Original: {str(res['original'])[:100]}...")
    print(f"Score: {res['score']}")
    print(f"Doc ID: {res['doc_id']}")
    print(f"Type: {res['type']}")
    print()

Summary: Customized diets are available to help treat and control various health issues in cats, such as alle...
Original: {'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'...
Score: 0.6065414547920227
Doc ID: 87c1181e-cbd8-47b1-9987-ce9f1ec2844c
Type: text

Summary: Here's a concise summary:

Commercial cat foods can impact your cat's condition, including haircoat,...
Original: {'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'...
Score: 0.6297948956489563
Doc ID: 5332e381-475f-45be-ad3b-1b66e8daa67e
Type: text

Summary: Prescription diets are recommended for cats with specific medical conditions, such as heart disease,...
Original: {'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'...
Score: 0.6425747871398926
Doc ID: 1ba7e737-d441-4c4e-8116-e7e1fa160057
Type: text

Summary: Cats may not self-regulate their diet due to taste preference

In [17]:
# For summaries
summary_docs = vectorstore_summaries.get(limit=5, include=['metadatas'])
print("Summary doc IDs:", [m.get('doc_id') for m in summary_docs['metadatas']])

# For originals
original_docs = vectorstore_original_texts.get(limit=5, include=['metadatas'])
print("Original doc IDs:", [m.get('doc_id') for m in original_docs['metadatas']])

Summary doc IDs: ['e330983d-355b-4b67-98bc-ef44e43234d2', 'e280da7e-2142-43a7-9827-6e86ec4ee960', '055889bb-c09b-4ee2-9c88-89fbd9562d2a', '49223229-9382-4f49-9f77-33314de55e56', 'c2f08628-460c-463f-88e2-0ed7c6dad596']
Original doc IDs: ['e330983d-355b-4b67-98bc-ef44e43234d2', 'e280da7e-2142-43a7-9827-6e86ec4ee960', '055889bb-c09b-4ee2-9c88-89fbd9562d2a', '49223229-9382-4f49-9f77-33314de55e56', 'c2f08628-460c-463f-88e2-0ed7c6dad596']


In [18]:
# Try fetching a known doc_id directly
doc_id = "e330983d-355b-4b67-98bc-ef44e43234d2"
result = vectorstore_original_texts.get(ids=[doc_id], include=['documents', 'metadatas'])
print(result)

{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['documents', 'metadatas'], 'data': None, 'metadatas': []}


In [19]:
# Access the underlying Chroma collection
collection = vectorstore_original_texts._collection  # This is a chromadb Collection object

# Now use the chromadb get method
result = collection.get(ids=[doc_id], include=['documents', 'metadatas'])
print(result)

{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['documents', 'metadatas'], 'data': None, 'metadatas': []}


In [23]:
doc_id = "e330983d-355b-4b67-98bc-ef44e43234d2"
collection = vectorstore_original_texts._collection
result = collection.get(ids=[doc_id], include=['documents', 'metadatas'])
print(result)

{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['documents', 'metadatas'], 'data': None, 'metadatas': []}
